## 1. 如何设置网络初始值
一般总是使用服从（截断）高斯或均匀分布的随机值，具体是高斯还是均匀分布影响不大，但是也没有详细的研究。

但是，初始值的大小会对优化结果和网络的泛化能力产生较大的影响。

一些启发式初始化策略通常是根据输入与输出的单元数来决定初始权重的大小，比如 Glorot and Bengio (2010) 中建议建议使用的标准初始化，其中 m 为输入数，n 为输出数

$$W_{i,j} \sim U(-\sqrt{\frac{6}{m+n}}, \sqrt{\frac{6}{m+n}})$$

还有一些方法推荐使用随机正交矩阵来初始化权重 (Saxe et al., 2013)。

## 2. 为什么推荐使用高斯分布？
当我们由于缺乏关于某个实数上分布的先验知识而不知道该选择怎样的形式时，正态分布是默认的比较好的选择，其中有两个原因：
   1. 我们想要建模的很多分布的真实情况是比较接近正态分布的。**中心极限定理（central limit theorem）**说明很多独立随机变量的和近似服从正态分布。这意味着在实际中，很多复杂系统都可以被成功地建模成正态分布的噪声，即使系统可以被分解成一些更结构化的部分。
   2. 在具有相同方差的所有可能的概率分布中，正态分布在实数上具有最大的不确定性。因此，我们可以认为正态分布是对模型加入的先验知识量最少的分布。
   
正态分布有单边量和多变量形式， 
- 单变量  
对于单边量$x \in R$，高斯分布的参数为均值$\mu \in R$和方差$\sigma^2>0$，其概率密度函数
$$p(x|u,\sigma^2) = \frac{1}{\sqrt{2\pi \sigma^2}}exp\left\{-\frac{(x-\mu)^2}{2\sigma^2}\right\}$$
其均值和方差分别为$\mu$和$\sigma^2$   

- 多变量   
对于$d$维向量$\boldsymbol{x}$，多元高斯分布的参数为$d$维向量$\boldsymbol{\mu}$和$d \times d$的对称正定协方差矩阵$\boldsymbol{\Sigma}$，概率密度函数 
$$p(\boldsymbol{x}|\boldsymbol{\mu}, \boldsymbol{\Sigma}) = \frac{1}{\sqrt{(2\pi)^d det(\boldsymbol{\Sigma})}}exp\left\{-\frac{1}{2}(\boldsymbol{x}-\boldsymbol{\mu})^T\boldsymbol{\Sigma}^{-1}(\boldsymbol{x}-\boldsymbol{\mu})\right\}$$